In [1]:
from connector.replay import Replay, State, Team
from preprocessing    import remove_menu_and_pause
from preprocessing    import who_scored
from preprocessing    import game_state_to_numpy

from typing import Optional

import numpy as np

In [2]:
def prepare_train_data(episodes, stack_frames=4):
    # Take resultative episodes as training data
    train_x, train_y = [], []
    for episode in episodes:
        scorer = who_scored(episode)
        if scorer is not None:
            if scorer == Team.Blue:
                transform_blue_pivotal(episode)
            x, y = zip(*[game_state_to_numpy(e, scorer) for e in episode])

            # If there was no target player - ignore
            if y is not None:
                train_x.append(x)
                train_y.append(y)
                
    # Stack n-frames
    x = []
    y = []
    for episode_i in range(len(train_x)):
        episode_x      = train_x[episode_i]
        episode_y      = train_y[episode_i]
        
        # Stacked samples
        train_sample = [np.concatenate(episode_x[0]).ravel() for _ in range(stack_frames)]
        
        x.append(np.concatenate(train_sample).ravel())
        y.append(np.array(episode_y[0]))
        
        for frame_i in range(1, len(episode_x)):
            frame_x = episode_x[frame_i]
            frame_y = episode_y[frame_i]
            
            # Move stack
            for i in range(1, stack_frames):
                train_sample[i] = train_sample[i - 1]
            train_sample[0] = np.concatenate(frame_x).ravel()
            
            x.append(np.concatenate(train_sample).ravel())
            y.append(np.array(frame_y))
            
    return x, y

In [3]:
def transform_blue_pivotal(frames) -> Replay:
    """
    Inplace transformation of disk positions and velocities based on the pivotal team side.
    Inverse x-axis basically.
    
    frames: a list of replay frames
    """
    for frame in frames:
        if frame.players == None:
            continue
            
        for player in frame.players:
            # Inverse position and velocity along x-axis
            player.disc.x  = -player.disc.x
            player.disc.vx = -player.disc.vx
            
            # Swap left and right actions
            temp            = player.input[2]
            player.input[2] = player.input[3]
            player.input[3] = temp
            
        # Inverse position and velocity of the ball along x-axis
        frame.ball.x  = -frame.ball.x
        frame.ball.vx = -frame.ball.vx

In [4]:
# Parse replay and etract game epsiodes
names, replay = Replay(open("replays/cups/edition7/2018-12-02.bin", mode="rb").read())
episodes = remove_menu_and_pause(replay)
t1, t2 = prepare_train_data(episodes, 1)

In [ ]:
import lightgbm as lgb

In [ ]:
imitator = lgb.LGBMClassifier()